SETUP THE VARIABLES AND DO THE IMPORTS

In [1]:
import os
import dao
import pandas as pd
import prediction
import backtesting
import prediction
import config
from datetime import datetime
import warnings
import utils
import simulation
import concurrent.futures
warnings.simplefilter(action='ignore', category=FutureWarning)
current_date = datetime.now().strftime("%Y%m%d")
current_date_time = datetime.now().strftime("%Y%m%d%H%M%S")

Select the run settings

In [2]:
# stock equity research filetype pdf
run_type = "BOUGHT"  # options --> GENERIC, BOUGHT, SIM
initial_funds = 1000  # funds per stock for simulation
# current_date = "20241124" #comment out for latest date run

declare the variables

In [3]:
portfolio_exec_flag = False
exploratory_exec_flag = False

data_dump_path = config.path_data_dump + run_type + "\\"
static_files_path = config.path_static_folder
summaries_files_path = config.path_summaries_folder + run_type + "\\"
stocks = []

Get the stock list based on run type

In [4]:
if run_type == "GENERIC":
    count = 50
    stock_filters = [config.WeekGainers52, config.WeekLosers52] #MostActive, Gainers, Losers, TrendingNow, WeekGainers52, WeekLosers52
    for stock_filter in stock_filters:
        stocks += dao.get_yahoo_finance_stock_list(
            stock_filter+config.stock_list_appender, count)
    stocks = pd.unique(stocks).tolist()
    exploratory_exec_flag = True
    
elif run_type == "BOUGHT":
    portfolio_json_file = "real_portfolio.json"
    portfolio_file = static_files_path + portfolio_json_file
    portfolio_stocks = dao.read_json(portfolio_file)
    for item in portfolio_stocks:
        stocks.append(item['ticker'])
    portfolio_exec_flag = True
    
elif run_type == "SIM":
    portfolio_json_file = "sim_portfolio.json"
    portfolio_file = static_files_path + portfolio_json_file
    portfolio_stocks = dao.read_json(portfolio_file)
    for item in portfolio_stocks:
        stocks.append(item['ticker'])
    portfolio_exec_flag = True

print(stocks)

['AAPL', 'NVDA', 'TSLA', 'AMZN', 'AMD', 'NFLX', 'MSFT', 'GOOGL', 'DIS', 'META', 'SMR', 'WGS', 'MSTR', 'SMTC', 'CSAN', 'SBS', 'DB', 'FIVE', 'TMC', 'RDDT', 'SYM']


Get data for each stock and create predictions

In [5]:
# Function to process each stock
def process_stock(stock, data_dump_path, run_type, current_date):
    print(f"{stock} processing for {current_date}")
    # Declare the files
    stats_file = data_dump_path + run_type + "_" + \
        stock + "_df_stats_" + str(current_date) + ".csv"
    pred_file = data_dump_path + run_type + "_" + \
        stock + "_df_pred_" + str(current_date) + ".csv"

    if not os.path.exists(pred_file):
        # Get the required data from Yahoo Finance
        df_5y = pd.DataFrame(data=dao.get_yahoo_finance_5y(stock))
        df_stats = pd.DataFrame(
            data=dao.get_yahoo_finance_key_stats(stock)).iloc[0]

        # Write the stats data
        dao.write_csv(stats_file, df_stats)

        # Get the prediction
        df_pred = prediction.get_prediction(df_5y, df_stats)

        # Calculate the signal (Signal calculation logic)
        df_pred['Signal'] = df_pred['technical_analysis_buy_score'] + df_pred['technical_analysis_sell_score'] + \
            df_pred['fundamental_analysis_score'] + \
            df_pred['sentiment_analysis_score']

        # Convert the signal to text
        df_pred = prediction.convert_signal_to_text(df_pred)

        # Write the prediction data
        dao.write_csv(pred_file, df_pred)

    print(f"{stock} processed for {current_date}")

# Main function to execute multi-threading
def process_stocks_in_parallel(stocks, data_dump_path, run_type, current_date):
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(lambda stock: process_stock(
            stock, data_dump_path, run_type, current_date), stocks)


# Call the main function to process the stocks in parallel
process_stocks_in_parallel(stocks, data_dump_path, run_type, current_date)

ERROR:root:Error in get_yahoo_finance_5y for AAPL
ERROR:root:Error in get_yahoo_finance_5y for NVDA
ERROR:root:Error in get_yahoo_finance_5y for TSLA
ERROR:root:Error in get_yahoo_finance_5y for AMZN
ERROR:root:Error in get_yahoo_finance_5y for AMD
ERROR:root:Error in get_yahoo_finance_5y for NFLX
ERROR:root:Error in get_yahoo_finance_5y for MSFT
ERROR:root:Error in get_yahoo_finance_5y for GOOGL
ERROR:root:Error in get_yahoo_finance_5y for DIS
ERROR:root:Error in get_yahoo_finance_5y for META
ERROR:root:Error in get_yahoo_finance_5y for SMR
ERROR:root:Error in get_yahoo_finance_5y for WGS
ERROR:root:Error in get_yahoo_finance_5y for MSTR
ERROR:root:Error in get_yahoo_finance_5y for SMTC
ERROR:root:Error in get_yahoo_finance_5y for CSAN
ERROR:root:Error in get_yahoo_finance_5y for SBS
ERROR:root:Error in get_yahoo_finance_5y for DB
ERROR:root:Error in get_yahoo_finance_5y for FIVE
ERROR:root:Error in get_yahoo_finance_5y for TMC
ERROR:root:Error in get_yahoo_finance_5y for RDDT
ERROR:r

AAPL processing for 20250726
NVDA processing for 20250726
TSLA processing for 20250726
AMZN processing for 20250726
AMD processing for 20250726
NFLX processing for 20250726
MSFT processing for 20250726
GOOGL processing for 20250726
DIS processing for 20250726
META processing for 20250726
SMR processing for 20250726
WGS processing for 20250726
MSTR processing for 20250726
SMTC processing for 20250726
CSAN processing for 20250726
SBS processing for 20250726
DB processing for 20250726
FIVE processing for 20250726
TMC processing for 20250726
RDDT processing for 20250726
SYM processing for 20250726


Backtest the predictions and create the summary

In [6]:
# GET THE SUMMARY FILES IF IT EXISTS ELSE CREATE NEW
prediction_summary_file = summaries_files_path + run_type + \
    "_prediction_summary_"+str(current_date)+".csv"
prediction_summary_df = utils.get_summary_file(name=prediction_summary_file)

# LOOP OVER STOCKS AND PROCESS EACH STOCK
for stock in stocks:
    print(stock)

    # DECLARE THE FILES
    pred_file = data_dump_path + run_type + "_" + \
        stock + "_df_pred_" + str(current_date) + ".csv"
    tested_file = data_dump_path + run_type + "_" + \
        stock + "_df_tested_" + str(current_date) + ".csv"

    # READ LOAD THE PRED FILE
    file_to_test_df = pd.read_csv(pred_file)

    # BACKTEST THE SIGNAL
    df_tested, cumulative_return, sharpe_ratio, sortino_ratio, max_drawdown, calmar_ratio = backtesting.backtest(
        file_to_test_df)

    # WRITE THE PREDICTION BACKTESTED DATA
    dao.write_csv(tested_file, df_tested)

    # GET THE SUMMARY
    start_iloc = 1
    end_iloc = 30
    signals = df_tested["Signal_Text"].iloc[start_iloc:end_iloc +
                                            1].reset_index(drop=True)
    signal_text, signal_number = prediction.get_weighted_signal(signals)
    prediction_summary_df = pd.concat([prediction_summary_df,
                                       pd.DataFrame([[
                                           current_date,
                                           stock,
                                           signal_number,
                                           signal_text,
                                           cumulative_return,
                                           sharpe_ratio,
                                           sortino_ratio,
                                           max_drawdown,
                                           calmar_ratio
                                       ]],
                                           columns=prediction_summary_df.columns)],
                                      ignore_index=True)
    prediction_summary_df.drop_duplicates(inplace=True)

    # WRITE THE SUMMARY DATA
    dao.write_csv(prediction_summary_file, prediction_summary_df)

AAPL
NVDA
TSLA
AMZN
AMD
NFLX
MSFT
GOOGL
DIS
META
SMR
WGS
MSTR
SMTC
CSAN
SBS
DB
FIVE
RDDT
SYM


Simulate exploratory trades from scratch

In [7]:
if exploratory_exec_flag:
    for stock in stocks:
        print(stock)
        # DECLARE THE FILES
        pred_file = data_dump_path + run_type + "_" + \
            stock + "_df_pred_" + str(current_date) + ".csv"
        exploratory_simulation_file = data_dump_path + run_type + "_" + \
            stock + "_exploratory_simulation_" + str(current_date) + ".json"

        # PREPARE THE SIM DATA
        file_to_sim_df = pd.read_csv(pred_file)
        data_to_sim_df = file_to_sim_df[[
            'High', 'Low', 'Close', 'Date', 'TICKER', 'Signal_Text']]

        # INITIALISE THE SIM VALUES
        start_iloc = 1  # 0 is the latest day
        end_iloc = 31  # 29 is the 30th day

        # DO THE SIM
        simulation_df = simulation.simulate_exploratory_trading(
            data_to_sim_df, start_iloc, end_iloc, initial_funds)

        # WRITE THE DATA
        dao.write_json(exploratory_simulation_file, simulation_df)
else:
    print('exploratory_exec_flag set to: '+str(exploratory_exec_flag))

exploratory_exec_flag set to: False


Summarise the exploratory simulation

In [8]:
if exploratory_exec_flag:
    # GET THE SUMMARY FILES IF IT EXISTS ELSE CREATE NEW
    exploratory_simulation_summary_file = summaries_files_path + run_type + \
        "_exploratory_simulation_summary_"+str(current_date)+".csv"
    exploratory_simulation_summary_df = utils.get_simulation_file(
        exploratory_simulation_summary_file)

    # LOOP OVER STOCKS AND PROCESS EACH STOCK
    for stock in stocks:
        print(stock)

        # DECLARE THE FILES
        exploratory_simulation_file = data_dump_path + run_type + "_" + \
            stock + "_exploratory_simulation_" + str(current_date) + ".json"

        # READ FILE
        file_to_json = dao.read_json(exploratory_simulation_file)

        positions = []
        for json in file_to_json['transactions']:
            positions.append(json['position'])
        series = pd.Series(positions)
        days_to_consider = len(positions)

        signal_text, signal_number = prediction.get_weighted_signal(series)

        exploratory_simulation_summary_df = pd.concat([exploratory_simulation_summary_df,
                                                       pd.DataFrame([[
                                                           current_date,
                                                           stock,
                                                           file_to_json['closing_stock_price'],
                                                           file_to_json['final_cash_balance'],
                                                           file_to_json['unrealized_gains_losses'],
                                                           file_to_json['unrealized_gain_loss_%'],
                                                           file_to_json['units_held'],
                                                           file_to_json['average_price_per_unit'],
                                                           signal_text,
                                                           signal_number
                                                       ]],
                                                           columns=exploratory_simulation_summary_df.columns)],
                                                      ignore_index=True)
        exploratory_simulation_summary_df.drop_duplicates(inplace=True)

        dao.write_csv(exploratory_simulation_summary_file,
                      exploratory_simulation_summary_df)
else:
    print('exploratory_exec_flag set to: '+str(exploratory_exec_flag))

exploratory_exec_flag set to: False


Simulate current portfolio trades

In [9]:
if portfolio_exec_flag:
    portfolio_file = static_files_path + portfolio_json_file
    portfolio_stocks = dao.read_json(portfolio_file)

    for stock in portfolio_stocks:
        print(stock)

        # DECLARE THE FILES
        pred_file = data_dump_path + run_type + "_" + \
            stock['ticker'] + "_df_pred_" + str(current_date) + ".csv"
        portfolio_simulation_file = data_dump_path + run_type + "_" + \
            stock['ticker'] + "_portfolio_simulation_" + \
            str(current_date) + ".json"

        # PREPARE THE SIM DATA
        file_to_sim_df = pd.read_csv(pred_file)
        data_to_sim_df = file_to_sim_df[[
            'High', 'Low', 'Close', 'Date', 'TICKER', 'Signal_Text']]

        # GET ILOCS
        start_iloc = 1
        end_iloc = data_to_sim_df.index[data_to_sim_df['Date'] == stock['date']].tolist()[
            0]
        units_held = stock['units']
        avg_price = stock['average_buying_price']

        # DO THE SIM
        simulation_df = simulation.simulate_portfolio_trades(
            data_to_sim_df, start_iloc, end_iloc, initial_funds, units_held, avg_price)

        # WRITE THE DATA
        dao.write_json(portfolio_simulation_file, simulation_df)
else:
    print('portfolio_exec_flag set to: '+str(portfolio_exec_flag))

{'ticker': 'AAPL', 'date': '2024-12-02', 'units': 0.044192, 'average_buying_price': 52.72}
{'ticker': 'NVDA', 'date': '2024-12-04', 'units': 0.0415, 'average_buying_price': 138.55}
{'ticker': 'TSLA', 'date': '2024-11-19', 'units': 0.002977, 'average_buying_price': 335.91}
{'ticker': 'AMZN', 'date': '2024-11-19', 'units': 0.005041, 'average_buying_price': 198.37}
{'ticker': 'AMD', 'date': '2024-12-04', 'units': 0.093372, 'average_buying_price': 137.51}
{'ticker': 'NFLX', 'date': '2024-11-19', 'units': 0.00119, 'average_buying_price': 840.34}
{'ticker': 'MSFT', 'date': '2024-11-19', 'units': 0.002416, 'average_buying_price': 413.91}
{'ticker': 'GOOGL', 'date': '2024-11-19', 'units': 0.005736, 'average_buying_price': 174.34}
{'ticker': 'DIS', 'date': '2024-11-19', 'units': 0.008897, 'average_buying_price': 112.4}
{'ticker': 'META', 'date': '2024-11-19', 'units': 0.00502, 'average_buying_price': 597.61}
{'ticker': 'SMR', 'date': '2024-11-19', 'units': 0.0741, 'average_buying_price': 26.99}

Summarise the potrfolio simulation

In [10]:
if portfolio_exec_flag:
    # GET THE SUMMARY FILES IF IT EXISTS ELSE CREATE NEW
    portfolio_simulation_summary_file = summaries_files_path + run_type + \
        "_portfolio_simulation_summary_"+str(current_date)+".csv"
    portfolio_simulation_summary_df = utils.get_simulation_file(
        portfolio_simulation_summary_file)

    # LOOP OVER STOCKS AND PROCESS EACH STOCK
    for stock in stocks:
        print(stock)

        # DECLARE THE FILES
        portfolio_simulation_file = data_dump_path + run_type + "_" + \
            stock + "_portfolio_simulation_" + str(current_date) + ".json"

        # READ FILE
        file_to_json = dao.read_json(portfolio_simulation_file)

        positions = []
        for json in file_to_json['transactions']:
            positions.append(json['position'])
        series = pd.Series(positions)
        days_to_consider = len(positions)

        signal_text, signal_number = prediction.get_weighted_signal(series)

        portfolio_simulation_summary_df = pd.concat([portfolio_simulation_summary_df,
                                                     pd.DataFrame([[
                                                         current_date,
                                                         stock,
                                                         file_to_json['closing_stock_price'],
                                                         file_to_json['final_cash_balance'],
                                                         file_to_json['unrealized_gains_losses'],
                                                         file_to_json['unrealized_gain_loss_%'],
                                                         file_to_json['units_held'],
                                                         file_to_json['average_price_per_unit'],
                                                         signal_text,
                                                         signal_number
                                                     ]],
                                                         columns=portfolio_simulation_summary_df.columns)],
                                                    ignore_index=True)
        portfolio_simulation_summary_df.drop_duplicates(inplace=True)

        dao.write_csv(portfolio_simulation_summary_file,
                      portfolio_simulation_summary_df)
else:
    print('portfolio_exec_flag set to: '+str(portfolio_exec_flag))

AAPL
NVDA
TSLA
AMZN
AMD
NFLX
MSFT
GOOGL
DIS
META
SMR
WGS
MSTR
SMTC
CSAN
SBS
DB
FIVE
RDDT
SYM
